In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001DC718C9790>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001DC718CAED0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Muskan and I am a Chief AI Engineer")])

AIMessage(content="Hello Muskan,\n\nIt's a pleasure to meet you! As a Chief AI Engineer, I imagine you have a fascinating and challenging role. What are you currently working on that's most exciting to you? \n\nI'm eager to learn more about your work in the field of AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 23, 'total_tokens': 88, 'completion_time': 0.118181818, 'prompt_time': 0.000919768, 'queue_time': 0.02335016, 'total_time': 0.119101586}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-dc3312bb-dca7-436c-999d-a70d1ef10bb7-0', usage_metadata={'input_tokens': 23, 'output_tokens': 65, 'total_tokens': 88})

In [4]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hi , My name is Muskan and I am a Chief AI Engineer"),
    AIMessage(content="Hi Muskan, it's nice to meet you! That's a very impressive title. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days?"),
    HumanMessage(content="Hey Whats my name and what do I do?")
])

AIMessage(content="You are Muskan, and you are a Chief AI Engineer. \n\nIs there anything else you'd like to tell me about yourself or your work? I'm eager to learn more! 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 82, 'total_tokens': 127, 'completion_time': 0.081818182, 'prompt_time': 0.003277464, 'queue_time': 0.031417213, 'total_time': 0.085095646}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-614deaad-5b47-4772-8f26-40ce7a0a8420-0', usage_metadata={'input_tokens': 82, 'output_tokens': 45, 'total_tokens': 127})

### *Message History*
- We can use a message History class to wrap our model it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input.

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_messages_history=RunnableWithMessageHistory(model,get_session_history)

In [6]:
config={"configurable":{"session_id":"chat1"}}

In [7]:
response=with_messages_history.invoke(
    [HumanMessage(content="Hi , My name is Muskan and I am a Chief AI Engineer")],
    config=config
)

In [8]:
response.content

"Hello Muskan, it's nice to meet you!\n\nBeing a Chief AI Engineer is an exciting role. What kind of projects are you working on these days?  \n\nI'm eager to learn more about your work in the field of AI.\n"

In [9]:
with_messages_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Muskan.  😊 \n\nI remember that from our introduction! \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 92, 'total_tokens': 113, 'completion_time': 0.038181818, 'prompt_time': 0.002616204, 'queue_time': 0.010873054, 'total_time': 0.040798022}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-28949177-f085-464e-a165-c49bee2211d1-0', usage_metadata={'input_tokens': 92, 'output_tokens': 21, 'total_tokens': 113})

In [10]:
## change the config-->session id
config1={"configurable":{"session_id":"chat1"}}
response=with_messages_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is Muskan.  \n\nHow can I help you further? 😊 \n'

In [11]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_messages_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As a large language model, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to learn it!\n"

In [12]:
response=with_messages_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you.  😊  \n\nIs there anything I can help you with today?\n"

In [13]:
response=with_messages_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John, you told me earlier!  \n\nDo you need me to remember anything else for you? 😄  \n'

## **Prompt Templates**

In [14]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the best of your ability"),
         MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [15]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Muskan")]})

AIMessage(content="Hi Muskan, it's nice to meet you!  \n\nI'm happy to help with any questions you have. Just ask away! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 31, 'total_tokens': 67, 'completion_time': 0.065454545, 'prompt_time': 0.000355359, 'queue_time': 0.013419871, 'total_time': 0.065809904}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-12cb10a2-cde1-48a9-ae2d-4d0e510956ad-0', usage_metadata={'input_tokens': 31, 'output_tokens': 36, 'total_tokens': 67})

In [16]:
with_messages_history=RunnableWithMessageHistory(chain,get_session_history)

In [17]:
config = {"configurable": {"session_id":"chat3"}}
response=with_messages_history.invoke(
    [HumanMessage(content="Hi My name is Muskan")],
    config=config
)

response.content

"Hi Muskan! 👋  \n\nIt's nice to meet you. \n\nWhat can I do for you today? I'm here to answer your questions and help in any way I can. 😊  \n\n"

In [18]:
## Add more complexity

prompt=ChatPromptTemplate.from_messages(
    [
        (
         "system",
         "You are a helpful assistant.Answer all the question to the best of your ability in {language}"
        ),
         MessagesPlaceholder(variable_name="messages"),
    ]
)

chain=prompt|model

In [21]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Muskan")],"language":"Hindi"})
response.content

'नमस्ते मुस्कन! 😊 \n\nमुझे बहुत खुशी है कि आप मुझसे बात कर रहे हैं। \n\nआप मुझसे क्या पूछना चाहेंगे? मैं अपनी पूरी कोशिश करूँगा कि आपकी सभी प्रश्नों का जवाब सही और मददगार तरीके से दूँ। \n\n'

In [22]:
with_messages_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [23]:
config = {"configurable":{"session_id": "chat4"}}
response=with_messages_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Muskan")],"language":"Hindi"},
    config=config
)
response.content

'नमस्ते, मस्कन! 😊 \n\nमैं आपकी मदद के लिए यहाँ हूँ। आप मुझसे कुछ भी पूछ सकते हैं, और मैं अपने ज्ञान के अनुसार जवाब देने की पूरी कोशिश करूँगा।  \n\nआप क्या जानना चाहती हैं? \n\n'

In [24]:
response = with_messages_history.invoke(
    {"messages":[HumanMessage(content="What my name?")],"language":"Hindi"},
    config=config,

)

In [25]:
response.content

'आपका नाम मस्कन है। 😊  \n'

##   **Managing the Conversation History**

"""
# What is trim_message in LangChain?

The trim_message function in LangChain is used to clean and shorten a message. This is particularly useful when dealing with verbose outputs or messages from language models that need to be processed or displayed in a concise format.

---

## Key Features of trim_message

1. *Removes Redundancies*:
   Trims excessive details or unnecessary information from the message.

2. *Length Management*:
   Ensures the message stays within a defined character limit, making it suitable for use in UIs or logs.

3. *Enhances Readability*:
   Keeps the message focused on its core content by removing extraneous details.

4. *Customizable*:
   Supports configurable trimming logic to meet specific requirements.

---

## Usage Example

Here’s a basic example of how trim_message might be used:

```python
from langchain_utils import trim_message

# Example message
long_message = "This is a very long message that contains a lot of unnecessary details and information that can be trimmed for better readability."

# Trim the message to a maximum of 50 characters
trimmed_message = trim_message(long_message, max_length=50)

print("Trimmed Message:", trimmed_message)

In [26]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

d:\Generative AI\Chatbot-With-Conversation-Using-Langchain\generative4\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Generative AI\Chatbot-With-Conversation-Using-Langchain\generative4\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\md199\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run P

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [27]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What ice cream do I like")],
        "language": "English"
    }
)
response.content

"As an AI, I don't have access to your personal information, including your ice cream preferences.  \n\nDo you have a favorite flavor you'd like to tell me about? 😊🍦  \n\n"

In [28]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What math problem did i ask")],
        "language": "English"
    }
)
response.content

'You asked "what\'s 2 + 2" 🧮  \n\n\n\nLet me know if you\'d like to try another one! 😊\n'

In [29]:
## Lets wrap this in the message History
with_messages_history= RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)
config={"configurable":{"session_id":"chat5"}}

In [31]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What my name?")],
        "language": "English",
    },
    config=config
)
response.content

"As an AI, I don't have access to any personal information about you, including your name. 😊\n\nIs there anything else I can help you with?\n"

In [32]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What math problem did i ask")],
        "language": "English",
    },
    config=config
)
response.content

'You asked " whats 2 + 2".  \n\n\n\nLet me know if you have any other questions, I\'m happy to help! 😄\n'